In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

- max_depth – максимальная глубина.
По-умолчанию: 5
- min_samples_split – кол-во объектов в листе, чтобы его можно было разбить и превратить в узел.
По-умолчанию: 2
- max_leafs – максимальное количество листьев разрешенное для дерева.
По-умолчанию: 20

In [2]:
class MyTreeClf:
    def __init__(self, max_depth = 5, min_samples_split = 2, max_leafs  = 20, bins = None, criterion = "entropy"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0
        self.pred_sum = 0
        self.potential_leafs = 1
        self.bins = bins
        self.criterion = criterion
        self.fi = {}
       
    def _entropy(self, y):
        epsilon = 1e-12
        p0 = np.sum(y == 0) / len(y)
        p1 = np.sum(y == 1) / len(y)
        entropy = - (p0 * np.log2(p0 + epsilon) + p1 * np.log2(p1 + epsilon))
        return entropy

    def _gini(self, y):
        p0 = np.sum(y==0) / len(y)
        p1 = np.sum(y==1) / len(y)
        gini = 1 - (p0 ** 2 + p1**2)
        return gini

    def _calculate_criterion(self, y):
        if self.criterion == "entropy":
            return self._entropy(y)
        elif self.criterion == "gini":
            return self._gini(y)
        else:
            raise ValueError("criterion must be 'entropy' or 'gini'")
            
    def __repr__(self):
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"
        
    def _get_best_split(self, X, y):
       
        N, n_features = X.shape
        S_0 = self._calculate_criterion(y)
        max_IG = 0
        best_split = 0
        feature_index = 0
        
        for j in range(n_features):
            thresholds = self.global_thresholds_[j]
            for t in thresholds:
                mask_left = X[:, j] <= t
                mask_right = X[:, j] > t
                if mask_left.sum() == 0 or mask_right.sum() == 0:
                    continue
                X_r, y_r = X[mask_right], y[mask_right]
                X_l, y_l = X[mask_left], y[mask_left]
                S_r = self._calculate_criterion(y_r)
                S_l = self._calculate_criterion(y_l)
                N_r = len(y_r)
                N_l = len(y_l)
                IG = S_0 - N_r / N * S_r - N_l/N * S_l
                if IG > max_IG:
                    max_IG = IG
                    best_split = t
                    feature_index = j
                        
        return feature_index, max_IG, best_split

        
    def _build_tree(self, X_train, y_train, feature_names, depth = 0):
        
        stop_reasons = []
        if depth >= self.max_depth:
                stop_reasons.append("max_depth")
        
        if len(np.unique(y_train)) == 1:
            stop_reasons.append("pure_node")
    
        if len(y_train) == 1:
            stop_reasons.append("single_sample")
    
        if len(y_train) < self.min_samples_split:
            stop_reasons.append("min_samples_split")
    
        if self.potential_leafs >= self.max_leafs:
            stop_reasons.append("max_leafs")
            
        if stop_reasons:
            pred = np.mean(y_train)
            self.pred_sum += pred
            self.leafs_cnt += 1            
            impurity = self._calculate_criterion(y_train)           
            return {
                "type" : "leaf",
                "prediction" : pred,
                "n_samples" : len(y_train),
                "depth" : depth,
                }
        
        best_feature, ig, best_split = self._get_best_split(X_train, y_train)
        
        if ig <= 0:
            pred = np.mean(y_train)
            self.pred_sum += pred
            self.leafs_cnt += 1
            return {
                "type" : "leaf",
                "prediction" : pred,
                "n_samples" : len(y_train),
                "depth" : depth,
            }
        n_samples_node = len(y_train)
        fn = feature_names[best_feature]
        self.fi[fn] += n_samples_node / self.n_samples * ig
        
        self.potential_leafs += 1
        
        mask_left = X_train[:, best_feature] <= best_split
        mask_right = X_train[:, best_feature] > best_split
        X_r, y_r = X_train[mask_right], y_train[mask_right]
        X_l, y_l = X_train[mask_left], y_train[mask_left]
        left_subtree = self._build_tree(X_l, y_l, feature_names,depth +1)
        right_subtree = self._build_tree(X_r, y_r, feature_names, depth +1)
        
        return {
            "type" : "node",
            'feature' : best_feature,
            'split' : best_split,
            'feature_name' : fn,
            'depth' : depth,
            "n_samples" : n_samples_node,
            'leaf_left' : left_subtree,
            'leaf_right' : right_subtree,        
        }
        
    def fit(self, X, y):
        if self.max_leafs < 2:
            self.max_leafs = 2
        self.leafs_count = 0
        self.pred_sum = 0
        feature_names = X.columns.to_list()
        self.fi = {f : 0 for f in feature_names}
        self.n_samples = X.shape[0]
        
        X_train = X.to_numpy()
        y_train = y.to_numpy()
        
        self.global_thresholds_ = []
        for j in range(X_train.shape[1]):
            features = X_train[:, j]
            f = np.sort(np.unique(features))
            native_thresholds = (f[:-1] + f[1:]) / 2 
            if self.bins is None:
                thresholds = native_thresholds
            else:
                if self.bins >= len(native_thresholds):
                    thresholds = native_thresholds
                else:
                    thresholds = np.histogram(X_train[:, j], self.bins)[1][1:-1]
            self.global_thresholds_.append(thresholds)
            
        self.tree_ = self._build_tree(X_train, y_train, feature_names, depth = 0)
           
            
    def print_tree(self, node = None, path = "1", side = None):
        if node is None:
            node = self.tree_
        if node["type"] == "leaf":
            if side is not None:
                print(' '*node['depth'], f"{path}.{side} - {node['prediction']}")
            else:
                print(f"{path} - {node['prediction']}")
            return
        feature = node["feature_name"]
        split = node["split"]
        depth = node['depth']
        print(' '*depth, f"{path} - {feature} > {split}")
        self.print_tree(node["leaf_left"], path + ".1", side = "left") 
        self.print_tree(node["leaf_right"], path + ".2", side = "right")

    def predict_proba(self, X):
       
        X_test = X.to_numpy()
        n_samples = X.shape[0]
        probas = np.zeros(n_samples)
        for i in range(n_samples):
            node = self.tree_
            while node["type"] != "leaf":
                j = node['feature']
                predicat = node['split']
                if X_test[i, j] <= predicat:
                    node = node['leaf_left']
                else:
                    node = node['leaf_right']
            result = node['prediction']
            probas[i] = result
        return probas
        
    def predict(self, X):
        probas = self.predict_proba(X)
        pred = (probas > 0.5).astype(int)
        return pred
               

In [3]:
features = ['variance', 'skewness', 'curtosis', 'entropy', 'class']
data = pd.read_csv("data_banknote_authentication.txt", names = features)

In [547]:
X = data.drop("class", axis = 1)
y = data["class"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [548]:
tree = MyTreeClf( max_depth = 3, min_samples_split = 2, max_leafs  = 5, criterion = "entropy")
tree.fit(X_train, y_train)

In [445]:
tree.potential_leafs

5

In [446]:
tree.leafs_count

5

In [447]:
tree.pred_sum

2.7999939511616794

In [284]:
tree.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,

In [197]:
class MyTreeReg:
    def __init__(self, max_depth = 5, min_samples_split = 2, max_leafs = 20, bins = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
    def __repr__(self):
        return f"MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}"
    def _mse(self, y):
        y_mean = np.mean(y)
        return np.mean((y - y_mean) ** 2)
    def _get_best_split(self, X, y):
        
        N, n_features = X.shape
        I_p = self._mse(y)
        gain = 0
        split_value = 0
        col_index  = None
        for j in range(n_features):
            thresholds = self.global_thresholds_[j]
            for t in thresholds:
                mask_left = X[:, j] <= t
                mask_right = X[:, j] > t
                if mask_left.sum() == 0 or mask_right.sum() == 0:
                    continue
                X_r, y_r = X[mask_right], y[mask_right]
                X_l, y_l = X[mask_left], y[mask_left]
                
                I_r, I_l = self._mse(y_r), self._mse(y_l)
                N_r, N_l = len(y_r), len(y_l)
                
                IG = I_p - N_r / N * I_r - N_l/N * I_l
                if IG > gain:
                    gain = IG
                    split_value = t
                    col_index = j
                        
        return col_index, split_value, gain
        
    def _build_tree(self, X_train, y_train, feature_names, depth = 0):
        
        stop_reasons = []
        if depth >= self.max_depth:
                stop_reasons.append("max_depth")
        
        if len(np.unique(y_train)) == 1:
            stop_reasons.append("pure_node")
    
        if len(y_train) == 1:
            stop_reasons.append("single_sample")
    
        if len(y_train) < self.min_samples_split:
            stop_reasons.append("min_samples_split")
    
        if self.potential_leafs >= self.max_leafs:
            stop_reasons.append("max_leafs")
            
        if stop_reasons:
            pred = np.mean(y_train)
            self.pred_sum += pred
            self.leafs_cnt += 1                      
            return {
                "type" : "leaf",
                "prediction" : pred,
                "n_samples" : len(y_train),
                "depth" : depth,
                }
        
        best_feature, best_split, ig  = self._get_best_split(X_train, y_train)
        
        if ig <= 0:
            pred = np.mean(y_train)
            self.pred_sum += pred
            self.leafs_cnt += 1
            return {
                "type" : "leaf",
                "prediction" : pred,
                "n_samples" : len(y_train),
                "depth" : depth,
            }
        n_samples_node = len(y_train)
        fn = feature_names[best_feature]
        self.fi[fn] += n_samples_node / self.n_samples * ig
        self.potential_leafs += 1
        
        mask_left = X_train[:, best_feature] <= best_split
        mask_right = X_train[:, best_feature] > best_split
        X_r, y_r = X_train[mask_right], y_train[mask_right]
        X_l, y_l = X_train[mask_left], y_train[mask_left]
        left_subtree = self._build_tree(X_l, y_l, feature_names,depth +1)
        right_subtree = self._build_tree(X_r, y_r, feature_names, depth +1)
        
        return {
            "type" : "node",
            'feature' : best_feature,
            'split' : best_split,
            'feature_name' : fn,
            'depth' : depth,
            "n_samples" : n_samples_node,
            'leaf_left' : left_subtree,
            'leaf_right' : right_subtree,        
        }
    def fit(self, X, y):
        if self.max_leafs < 2:
            self.max_leafs = 2
        self.leafs_cnt = 0
        self.pred_sum = 0
        self.potential_leafs = 1
        feature_names = X.columns.to_list()
        self.n_samples = X.shape[0]
        self.fi = {f : 0 for f in feature_names}
        
        X_train = X.to_numpy()
        y_train = y.to_numpy()
        self.global_thresholds_ = []
        for j in range(X_train.shape[1]):
            features = X_train[:, j]
            f = np.sort(np.unique(features))
            native_thresholds = (f[:-1] + f[1:]) / 2 
            if self.bins is None:
                thresholds = native_thresholds
            else:
                if self.bins - 1 > len(native_thresholds):
                    thresholds = native_thresholds
                else:
                    thresholds = np.histogram(X_train[:, j], self.bins)[1][1:-1]
            self.global_thresholds_.append(thresholds)
            
        self.tree_ = self._build_tree(X_train, y_train, feature_names, depth = 0)
               
            
    def print_tree(self, node = None, path = "1", side = None):
        if node is None:
            node = self.tree_
        if node["type"] == "leaf":
            if side is not None:
                print(' '*node['depth'], f"{path}.{side} - {node['prediction']}")
            else:
                print(f"{path} - {node['prediction']}")
            return
        feature = node["feature_name"]
        split = node["split"]
        depth = node['depth']
        print(' '*depth, f"{path} - {feature} > {split}")
        self.print_tree(node["leaf_left"], path + ".1", side = "left") 
        self.print_tree(node["leaf_right"], path + ".2", side = "right")
    
    def predict(self, X):
        X_test = X.to_numpy()
        n_samples = X.shape[0]
        preds = np.zeros(n_samples)
        for i in range(n_samples):
            node = self.tree_
            while node["type"] != "leaf":
                j = node['feature']
                predicat = node['split']
                if X_test[i, j] <= predicat:
                    node = node['leaf_left']
                else:
                    node = node['leaf_right']
            result = node['prediction']
            preds[i] = result
        return preds

In [198]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
X, y = data['data'], data['target']
X,y = pd.DataFrame(X), pd.DataFrame(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = .3)

In [200]:
reg_tree = MyTreeReg(max_depth = 1, min_samples_split=1, max_leafs=2, bins = 8)

In [203]:
reg_tree.fit(X_train, y_train)

In [204]:
reg_tree.leafs_cnt

2

In [205]:
reg_tree.pred_sum

319.99922958397536

In [206]:
reg_tree.print_tree()

 1 - bmi > 0.004572166603000902
  1.1.left - 118.13559322033899
  1.2.right - 201.86363636363637


In [202]:
reg_tree.fi

{'age': 0,
 'sex': 0,
 'bmi': 1641.5769347351852,
 'bp': 0,
 's1': 0,
 's2': 0,
 's3': 0,
 's4': 0,
 's5': 0,
 's6': 0}